# KEN4157 - Reinforcement Learning - Blackjack Gymnasium
If you opened this notebook in Google Colab, we recommend to start by saving a copy of the notebook in your own Google Drive, such that you can save any of your changes and experiments.

## Installing & Importing Modules
We will start by installing importing some modules that will likely be useful for your assignment(s). This includes [Gymnasium](https://gymnasium.farama.org/), which is a framework containing many popular RL environments (a successor to the original Gym API from OpenAI).

In [1]:
import gymnasium as gym
import math
import numpy as np

from tqdm import tqdm

## Numpy Cheat Sheet
In case you are not already familiar with `numpy`, here are some examples of functions which may come in useful:

In [2]:
# Creates a fixed-size array of 5 entries, each initialised to 0.0
a = np.zeros(5)
# Creates a 5 x 6 array (i.e., matrix) initialised to all-zeros
b = np.zeros((5, 6))
# Creates a 2 x 3 x 5 (i.e., 3-dimensional) array initialised to all-zeros
c = np.zeros((2, 3, 5))

# Samples a number uniformly at random from [0, 1)
d = np.random.random()
# Generates an array with 5 numbers, each sampled uniformly at random from [0, 1)
e = np.random.random(5)

# Gives you the maximum number from the array `e`
f = np.max(e)
# Gives you the index that holds the maximum number in the array `e`
g = np.argmax(e)

# Clear all the variables we created above, purely for the purpose of examples, from memory
del a, b, c, d, e, f, g

## Setting up the Blackjack Environment
Here, we'll set up the Blackjack environment, and have a first look at how to interact with it according to the Gym API.

**Optional**:
- For a description and documentation of the environment, see: https://gymnasium.farama.org/environments/toy_text/blackjack/
- For the implementation of the environment, see: https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/toy_text/blackjack.py

In [3]:
# Load the environment. sab=True means we follow the rules exactly as described
# in the Sutton and Barto book
env = gym.make("Blackjack-v1", sab=True)

action_space = env.action_space
obs_space = env.observation_space

print("action space =", action_space)
# action space = Discrete(2): this means that we have two actions (0 and 1)

# Let's create an array with nicer names for the actions (taken from documentation)
ACTION_NAMES = ["Stick", "Hit"]

print("observation space =", obs_space)
# observation space = Tuple(Discrete(32), Discrete(11), Discrete(2)):
#  - The player's current sum can be any from a discrete set of 32 values
#  - Value of the dealer's face-up card can be any from a discrete set of 11 values
#  - Whether or not the player holds a usable ace is boolean (two possible values)

print(f"The first variable of observation space has {obs_space[0].n} possible values.")
print(f"The second variable of observation space has {obs_space[1].n} possible values.")
print(f"The third variable of observation space has {obs_space[2].n} possible values.")

action space = Discrete(2)
observation space = Tuple(Discrete(32), Discrete(11), Discrete(2))
The first variable of observation space has 32 possible values.
The second variable of observation space has 11 possible values.
The third variable of observation space has 2 possible values.


## Episodes with Random Policy
We'll show how to run a few episodes under a random policy. This will demonstrate how to interact with the Gym API and its most important functions and return values.

In [4]:
for _ in range(1):  # Run 5 episodes under random policy
    obs, info = env.reset()     # Resets environment to initial state, gives us observation of initial state
    done = False

    returns = 0.0   # collect returns for this episode (note: not doing any discounting)

    print(f"The initial game state is: {obs}")

    # The following loop will run one complete episode
    while not done:
        print(f"value of cards I'm holding = {obs[0]}")
        print(f"value of dealer's face-up card = {obs[1]}")
        if obs[2]:
            print("I have a usable ace")
        else:
            print("I do not have a usable ace")

        action = action_space.sample()  # This randomly selects one action from the action space

        print("I randomly chose to: ", ACTION_NAMES[action])

        next_obs, reward, terminated, truncated, info = env.step(action)    # Execute the action, observe successor state and reward
        print(f"{terminated=}, {truncated=}, {reward=}, {next_obs=}")
        done = terminated or truncated
        obs = next_obs
        returns = returns + reward

    print(f"The episode ended with returns = {returns}")
    print()

The initial game state is: (19, 3, 1)
value of cards I'm holding = 19
value of dealer's face-up card = 3
I have a usable ace
I randomly chose to:  Hit
terminated=False, truncated=False, reward=0.0, next_obs=(21, 3, 1)
value of cards I'm holding = 21
value of dealer's face-up card = 3
I have a usable ace
I randomly chose to:  Stick
terminated=True, truncated=False, reward=1.0, next_obs=(21, 3, 1)
The episode ended with returns = 1.0



## Table-Based Q-Learning
Below, you can write your own table-based Q-learning implementation.

In [5]:
# TODO: Q-learning
# Learn the best Q-values for any environment with Q-learning.
def q_learning(q_values: np.ndarray, episode_num: int, game_env: gym.Env, alpha: float=0.1, gamma: float=1.0, epsilon: float=0.1) -> np.ndarray:
    for _ in tqdm(range(episode_num)):
        obs, info = game_env.reset()
        while True:
            if np.random.random() < epsilon:
                action = action_space.sample()
            else:
                action = np.argmax(q_values[obs[0], obs[1], obs[2], :])
                
            next_obs, reward, terminated, truncated, info = game_env.step(action)
            
            if terminated:
                value = 0
            else:
                value = np.max(q_values[next_obs[0], next_obs[1], next_obs[2], :])
            
            q_values[obs[0], obs[1], obs[2], action] += alpha * (reward + gamma * value - q_values[obs[0], obs[1], obs[2], action])
            obs = next_obs
            if terminated or truncated:
                break
    return q_values
        

In [6]:
# Evalute the learned Q-values.
def eval_game(q_values: np.ndarray, episode_num: int, game_env: gym.Env) -> float:
    obs, info = game_env.reset()
    reward = 0
    for _ in tqdm(range(episode_num)):
        while True:
            action = np.argmax(q_values[obs[0], obs[1], obs[2], :])
            next_obs, reward, terminated, truncated, info = game_env.step(action)
            reward += reward
            obs = next_obs
            if terminated or truncated:
                break
    return reward / episode_num

In [7]:
q_values = np.zeros((obs_space[0].n, obs_space[1].n, obs_space[2].n, action_space.n))
episode_num = 10000
env = gym.make("Blackjack-v1", sab=True)

q_values = q_learning(q_values, episode_num, env, gamma=0.9, epsilon=1e-4)

100%|██████████| 10000/10000 [00:02<00:00, 4836.43it/s]


In [8]:
print(f"Average reward: {eval_game(q_values, 1000, env)}")

100%|██████████| 1000/1000 [00:00<00:00, 58838.52it/s]

Average reward: -0.002


In [9]:
np.min(q_values), np.max(q_values)

(-0.8702745079068737, 0.9942735831029776)